In [1]:
#%%writefile get_proxy.py
#%load get_bookinfo.py
#%run get_bookinfo.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor 

from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq

#
import requests
import pandas as pd
import numpy as np
import random
import re
import json
import csv


def get_proxy(which='free',now=False):
    which_dict={'kuai':"/home/pan/django_projects/wtb/ips_kuai.txt",
                'ihuan':"/home/pan/django_projects/wtb/ips_1000_ihuan.txt",
                'free':"/home/pan/django_projects/wtb/ips_free.txt",
                'OK':"/home/pan/django_projects/wtb/ips_OKs.txt"
               }
    ippos=[]
    with open(which_dict[which], 'r') as f:
            lines=f.readlines()
            for line in lines:
                line=line.strip()
                if len(line.split(":"))==2:
                    ippos.append(line)
    if now:
        sleep(1.5)
        return random.choice(ippos)
    #ippo
    old=[]
    #測試代理_________________________________________
    while True:
        fake_header = Headers(
            browser="chrome",  # Generate only Chrome UA
            os="win",  # Generate ony Windows platform
            headers=True  # generate misc headers
        )    
        UA=fake_header.generate()
        #UA = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        #已經測過的就不要
        ippo=random.choice(ippos)
        if len(old)/len(ippos) > 0.5:
            old=[]
        if ippo in old:
            continue
        else:
            old.append(ippo)
        #    
        ip=ippo.split(":")[0]
        proxies={
                "http": "http://"+ippo,
                "https": "http://"+ippo
                }  
        #print("test:"+ippo)    
        #clear_output(wait=True)
        try:
            test=['http://icanhazip.com/','https://myip.com.tw/','https://www.showmyipaddress.eu/']
            r = requests.get(random.choice(test),#test[1], 
                             headers=UA,
                             proxies=proxies,
                             timeout=4)
            r.encoding='utf8'
            #print(r.status_code)
            #print(r.text)
            #return
            #return 22
            #
            if(r.status_code == 200 and (ip in r.text or '&#46;'.join(ip.split('.')) in r.text) ):
                print("OK:"+which+"_"+ippo)
                #print(r.status_code)
                #print(r.text)
                r.close() 
                if which!='OK':
                    with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'a+') as f:
                        f.write(ippo+"\r\n")                
                return ippo
            else:#if not (r.status_code == 200 and ip in r.text):
                r.close()
                raise Exception('fail')
               
        except Exception as e:
            #print(str(e))
            #clear_output(wait=True) 
             
            sleep(1.5+random.uniform(0, 2))
            #sleep(0.1)
            continue
        #else:
        #    print(r.text)
        #    print("OK2:"+ip)
        #    r.close()       
        #    break    

In [ ]:
#kuai,ihuan,free
%time get_proxy('free')

In [2]:
if __name__ == '__main__':
    which_dict={
                #'kuai':"ips_kuai.txt",
                #'ihuan':"ips_1000_ihuan.txt",
                'free':'ips_free.txt',
                #'OK':'ips_OKs.txt'
               }
    for i in range(60):
        print(i)
        with ThreadPoolExecutor(max_workers=3) as executor:
            executor.map(get_proxy,which_dict.keys())

0
OK:free_1.0.139.36:8080
1
OK:free_110.232.64.171:4777
2
OK:free_182.52.74.76:34084
3
OK:free_24.172.82.94:53281
4
OK:free_5.59.141.152:61981
5
OK:free_178.222.250.223:46492
6
OK:free_128.199.188.231:47503
7
OK:free_1.20.97.4:52237
8
OK:free_68.183.38.201:8118
9
OK:free_118.173.232.215:43186
10
OK:free_128.199.188.231:47503
11
OK:free_181.113.225.198:53281
12
OK:free_128.199.188.231:47503
13
OK:free_118.174.233.41:41469
14
OK:free_119.235.53.135:32471
15
OK:free_24.172.225.122:53281
16
OK:free_182.52.51.53:60441
17
OK:free_128.199.217.244:47503
18
OK:free_91.187.113.205:53281
19
OK:free_181.117.176.236:61358
20
OK:free_206.189.36.13:47503
21
OK:free_118.175.207.205:61293
22
OK:free_117.240.59.115:36127
23
OK:free_103.243.132.62:30341
24
OK:free_103.55.160.55:80
25
OK:free_124.41.211.251:36205
26
OK:free_128.199.188.231:47503
27
OK:free_113.130.126.212:55763
28
OK:free_117.58.241.70:8080
29
OK:free_103.239.254.70:61967
30
OK:free_110.232.64.171:4777
31
OK:free_5.141.244.97:61288
32
OK: